# Company or brand name generator

This notebook can be used to generate a company or brand name based on nouns from different categories.
It lets you choose two of your favorite categories. Then, it proposes composite names made up from English words found in these categories. It saves the suggestions in an easy-to-read CSV file.

## Web scraping the word list
This part can be skipped if a noun list is already available in the correct format.
This is why the libraries are loaded twice in this notebook.

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from os import path
import requests

from bs4 import BeautifulSoup # for the actual web scraping
import math # for rounding
import re # regular expressions for filtering of links
from tqdm import tqdm # status bar

### Setup for scaping

In [2]:
# create the link as a composite
link_first_part = 'https://www.enchantedlearning.com'


url = link_first_part   + "/wordlist/"

# get the content from the link
response = requests.get(url, timeout=50)

# parse the content with BeautifulSoup with the html.parser method
soup = BeautifulSoup(response.content, "html.parser")

# check status code of the loaded page, it should return 200 to indicate an accessible website
print(response.status_code)

print(soup.prettify())

200
<html>
 <head>
  <title>
   Word Lists by Theme: Wordbanks - EnchantedLearning.com
  </title>
  <meta content="Vocabulary Word Lists by Theme." name="description"/>
  <meta content="Vocabulary Word Lists Wordlists Word Banks Word Banks Wordbanks Wordbank wordlist Theme wordbank word bank Halloween Vocabulary Wordlist  spelling  worksheets printables spelling writing colloring drawing boos to print Fall children's crafts, Crafts, children, kids, preschool. preschoolers crafts, kid's crafts, kindergarten crafts, crafts, K-3 crafts, kids activities crafts, crafts, K-6 crafts, K-12" name="keywords"/>
  <meta content='(PICS-1.1 "http://www.rsac.org/ratingsv01.html" l gen true comment "RSACi North America Server" by "jc@EnchantedLearning.com" on "1996.11.15T19:18-0500" r (n 0 s 0 v 0 l 0))' http-equiv="PICS-Label"/>
  <meta content="true" name="MSSmartTagsPreventParsing"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
 </head>
 <body bgcolor="#ffffff">
  <!--incl

In [3]:
r = re.compile("\/(wordlist)\/[a-z]{1,}\.(shtml)")

#word_links_def = list(filter(r.match, word_links))

word_links = []

for link in soup.find_all('a'):
    word_links.append(link.get('href'))
    word_links_def = list(filter(r.match, word_links))
word_links_def

['/wordlist/big.shtml',
 '/wordlist/country.shtml',
 '/wordlist/compoundwords.shtml',
 '/wordlist/animal.shtml',
 '/wordlist/happy.shtml',
 '/wordlist/adjectives.shtml',
 '/wordlist/adverbs.shtml',
 '/wordlist/alliteration.shtml',
 '/wordlist/amphibian.shtml',
 '/wordlist/animal.shtml',
 '/wordlist/opposites.shtml',
 '/wordlist/aprilfool.shtml',
 '/wordlist/art.shtml',
 '/wordlist/astronomy.shtml',
 '/wordlist/fall.shtml',
 '/wordlist/yard.shtml',
 '/wordlist/baseball.shtml',
 '/wordlist/bathroom.shtml',
 '/wordlist/beach.shtml',
 '/wordlist/big.shtml',
 '/wordlist/birds.shtml',
 '/wordlist/biomes.shtml',
 '/wordlist/birthday.shtml',
 '/wordlist/boat.shtml',
 '/wordlist/bodiesofwater.shtml',
 '/wordlist/body.shtml',
 '/wordlist/buildings.shtml',
 '/wordlist/office.shtml',
 '/wordlist/camping.shtml',
 '/wordlist/carnival.shtml',
 '/wordlist/carparts.shtml',
 '/wordlist/car.shtml',
 '/wordlist/castle.shtml',
 '/wordlist/cats.shtml',
 '/wordlist/chinesenewyear.shtml',
 '/wordlist/christma

In [4]:
indiv_links = []

for e in range(len(word_links_def)):
    indiv_links.append(link_first_part + word_links_def[e])

indiv_links # this will return a list showing all the main job category links

['https://www.enchantedlearning.com/wordlist/big.shtml',
 'https://www.enchantedlearning.com/wordlist/country.shtml',
 'https://www.enchantedlearning.com/wordlist/compoundwords.shtml',
 'https://www.enchantedlearning.com/wordlist/animal.shtml',
 'https://www.enchantedlearning.com/wordlist/happy.shtml',
 'https://www.enchantedlearning.com/wordlist/adjectives.shtml',
 'https://www.enchantedlearning.com/wordlist/adverbs.shtml',
 'https://www.enchantedlearning.com/wordlist/alliteration.shtml',
 'https://www.enchantedlearning.com/wordlist/amphibian.shtml',
 'https://www.enchantedlearning.com/wordlist/animal.shtml',
 'https://www.enchantedlearning.com/wordlist/opposites.shtml',
 'https://www.enchantedlearning.com/wordlist/aprilfool.shtml',
 'https://www.enchantedlearning.com/wordlist/art.shtml',
 'https://www.enchantedlearning.com/wordlist/astronomy.shtml',
 'https://www.enchantedlearning.com/wordlist/fall.shtml',
 'https://www.enchantedlearning.com/wordlist/yard.shtml',
 'https://www.enchan

In [5]:
# get all list names as itms in list

r1 = re.compile("\/(wordlist)\/")
r2 = re.compile("\.(shtml)")

column_names = [r1.sub("", item) for item in word_links_def]
column_names = [r2.sub("", item) for item in column_names] 

column_names

['big',
 'country',
 'compoundwords',
 'animal',
 'happy',
 'adjectives',
 'adverbs',
 'alliteration',
 'amphibian',
 'animal',
 'opposites',
 'aprilfool',
 'art',
 'astronomy',
 'fall',
 'yard',
 'baseball',
 'bathroom',
 'beach',
 'big',
 'birds',
 'biomes',
 'birthday',
 'boat',
 'bodiesofwater',
 'body',
 'buildings',
 'office',
 'camping',
 'carnival',
 'carparts',
 'car',
 'castle',
 'cats',
 'chinesenewyear',
 'christmas',
 'circus',
 'clothes',
 'colors',
 'columbus',
 'compoundwords',
 'computer',
 'constitution',
 'conjunctions',
 'container',
 'cooking',
 'cookingtools',
 'country',
 'dance',
 'dentist',
 'desserts',
 'dogs',
 'doctor',
 'dolch',
 'driving',
 'duos',
 'election',
 'elements',
 'emotions',
 'energy',
 'fall',
 'family',
 'farm',
 'fire',
 'fish',
 'flowers',
 'food',
 'foodweb',
 'people',
 'fruit',
 'furniture',
 'geography',
 'goodluck',
 'grammar',
 'groundhogday',
 'halloween',
 'happiness',
 'happy',
 'hats',
 'herbs',
 'holidays',
 'house',
 'householdd

### Access individual pages

In [6]:
# first list

url_sep = indiv_links[0]

# get the content from the link
response = requests.get(url_sep, timeout=50)

# parse the content with BeautifulSoup with the html.parser method
soup1 = BeautifulSoup(response.content, "html.parser")

# check status code of the loaded page, it should return 200 to indicate an accessible website
print(response.status_code)

print(soup1.prettify())

200
<!DOCTYPE html>
<html lang="en">
 <!--template:wordlist-->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="/apple-touch-icon-iphone.png" rel="apple-touch-icon"/>
  <meta content="Enchanted Learning" name="apple-mobile-web-app-title"/>
  <meta content="Enchanted Learning" name="application-name"/>
  <title>
   Ways to say "Big" - Vocabulary Word List - Enchanted Learning
  </title>
  <meta content='A vocabulary word list (word bank) of different ways to say "Big"!' name="description"/>
  <meta content="Adjectives, English, Grammar, Parts of Speech" name="keywords"/>
  <meta content='Ways to say "Big" - Vocabulary Word List' property="og:title"/>
  <meta content="https://www.EnchantedLearning.com/wordlist/big.shtml" property="og:url"/>
  <meta content="website" property="og:type"/>
  <meta content='A vocabulary word list (word bank) of different wa

In [7]:
# get words of first list

soup1.find_all('a')

list1=[]
for element in soup1.findAll("div", {"class":"wordlist-item"}):
    list1.append(element.get_text())
    
list1

['adult',
 'astronomical',
 'boundless',
 'brawny',
 'broad',
 'capacious',
 'chunky',
 'colossal',
 'considerable',
 'copious',
 'corpulent',
 'deep',
 'economy-sized',
 'elephantine',
 'enormous',
 'epic',
 'expansive',
 'extensive',
 'extra-large',
 'family-sized',
 'fat',
 'full',
 'full-sized',
 'gargantuan',
 'giant',
 'gigantic',
 'ginormous',
 'goodly',
 'grand',
 'great',
 'heaping',
 'heavy',
 'hefty',
 'Herculean',
 'huge',
 'hulking',
 'humongous',
 'husky',
 'immense',
 'imposing',
 'impressive',
 'infinite',
 'jumbo',
 'king-sized',
 'large',
 'limitless',
 'lofty',
 'long',
 'mammoth',
 'massive',
 'mega',
 'mighty',
 'miles-wide',
 'monstrous',
 'monumental',
 'mountainous',
 'multifarious',
 'multiplicity',
 'multitude',
 'numerous',
 'obese',
 'outsized',
 'overabundant',
 'oversized',
 'piled high',
 'plentiful',
 'plump',
 'portly',
 'powerful',
 'prodigious',
 'queen-sized',
 'roly-poly',
 'sizable',
 'spacious',
 'stout',
 'strapping',
 'substantial',
 'super',
 '

### Create list with all words

In [8]:
lists_single = []

for n in range(len(indiv_links)):
    url_sep = indiv_links[n]
    print(url_sep)
    response = requests.get(url_sep, timeout=50)
    print(response)
    soup = BeautifulSoup(response.content, "html.parser")
    #print(soup)
    ls = []
    for element in soup.findAll("div", {"class":"wordlist-item"}):
        ls.append(element.get_text())
        words = ls[-1]
        lists_single.append(words)
            #print(lists[-1])

#print(lists)

https://www.enchantedlearning.com/wordlist/big.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/country.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/compoundwords.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/animal.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/happy.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/adjectives.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/adverbs.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/alliteration.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/amphibian.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/animal.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/opposites.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/aprilfool.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/art.shtml
<Response [200]>
https://www.enchantedlearning.

<Response [200]>
https://www.enchantedlearning.com/wordlist/office.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/opposites.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/duos.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/people.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/adjectivesforpeople.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/pirate.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/plants.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/positivewords.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/postal.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/prepositions.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/presidentsday.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/pronouns.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/regularverbs.shtml
<Response

In [9]:
print(lists_single)

['adult', 'astronomical', 'boundless', 'brawny', 'broad', 'capacious', 'chunky', 'colossal', 'considerable', 'copious', 'corpulent', 'deep', 'economy-sized', 'elephantine', 'enormous', 'epic', 'expansive', 'extensive', 'extra-large', 'family-sized', 'fat', 'full', 'full-sized', 'gargantuan', 'giant', 'gigantic', 'ginormous', 'goodly', 'grand', 'great', 'heaping', 'heavy', 'hefty', 'Herculean', 'huge', 'hulking', 'humongous', 'husky', 'immense', 'imposing', 'impressive', 'infinite', 'jumbo', 'king-sized', 'large', 'limitless', 'lofty', 'long', 'mammoth', 'massive', 'mega', 'mighty', 'miles-wide', 'monstrous', 'monumental', 'mountainous', 'multifarious', 'multiplicity', 'multitude', 'numerous', 'obese', 'outsized', 'overabundant', 'oversized', 'piled high', 'plentiful', 'plump', 'portly', 'powerful', 'prodigious', 'queen-sized', 'roly-poly', 'sizable', 'spacious', 'stout', 'strapping', 'substantial', 'super', 'super-sized', 'sweeping', 'tall', 'thick', 'thick-set', 'titanic', 'towering',

### Create list of lists 

#### Alphabetical dictionary 
To get an idea of what kind of words will be in the generator, an alphabetical dictionary can be created to look at them, irrespective of their category.

In [10]:
alphabetical_ls = lists_single.copy()

In [11]:
alphabetical_ls

['adult',
 'astronomical',
 'boundless',
 'brawny',
 'broad',
 'capacious',
 'chunky',
 'colossal',
 'considerable',
 'copious',
 'corpulent',
 'deep',
 'economy-sized',
 'elephantine',
 'enormous',
 'epic',
 'expansive',
 'extensive',
 'extra-large',
 'family-sized',
 'fat',
 'full',
 'full-sized',
 'gargantuan',
 'giant',
 'gigantic',
 'ginormous',
 'goodly',
 'grand',
 'great',
 'heaping',
 'heavy',
 'hefty',
 'Herculean',
 'huge',
 'hulking',
 'humongous',
 'husky',
 'immense',
 'imposing',
 'impressive',
 'infinite',
 'jumbo',
 'king-sized',
 'large',
 'limitless',
 'lofty',
 'long',
 'mammoth',
 'massive',
 'mega',
 'mighty',
 'miles-wide',
 'monstrous',
 'monumental',
 'mountainous',
 'multifarious',
 'multiplicity',
 'multitude',
 'numerous',
 'obese',
 'outsized',
 'overabundant',
 'oversized',
 'piled high',
 'plentiful',
 'plump',
 'portly',
 'powerful',
 'prodigious',
 'queen-sized',
 'roly-poly',
 'sizable',
 'spacious',
 'stout',
 'strapping',
 'substantial',
 'super',
 '

In [12]:
alphabet_dict = dict()

for name in alphabetical_ls:
    character = name[:1].upper()
    if not character in alphabet_dict:
        alphabet_dict[character] = list()
    alphabet_dict[character].append(name)

In [13]:
alphabet_dict

{'A': ['adult',
  'astronomical',
  'Afghanistan',
  'Albania',
  'Algeria',
  'Andorra',
  'Angola',
  'Antigua and Barbuda',
  'Argentina',
  'Armenia',
  'Australia',
  'Austria',
  'Azerbaijan',
  'aftereffect',
  'afternoon',
  'afterthought',
  'airbag',
  'aircraft',
  'airline',
  'airplane',
  'airport',
  'airtight',
  'angelfish',
  'another',
  'anybody',
  'anyhow',
  'anyone',
  'anyplace',
  'anything',
  'anytime',
  'anyway',
  'anywhere',
  'applesauce',
  'armchair',
  'armhole',
  'armpit',
  'arrowhead',
  'artwork',
  'awestruck',
  'aardvark',
  'abalone',
  'African elephant',
  'African gray parrot',
  'African penguin',
  'African rock python',
  'African wild cat',
  'agouti',
  'airedale terrier',
  'Alaskan malamute',
  'albatross',
  'algae',
  'alligator',
  'alpaca',
  'American bison',
  'American cocker spaniel',
  'American crocodile',
  'American flamingo',
  'American golden plover',
  'American Robin',
  'American tree sparrow',
  'amoeba',
  'amph

#### List of lists 

In [14]:
list_topic = lists_single.copy()
len(list_topic)

33581

In [15]:
df_topic = pd.DataFrame(list_topic)
df_topic

,0
0,adult
1,astronomical
2,boundless
3,brawny
4,broad
5,capacious
6,chunky
7,colossal
8,considerable
9,copious


In [16]:
lists_single1 = []

for n in range(len(indiv_links)):
    
    url_sep = indiv_links[n]
    print(url_sep)
    response = requests.get(url_sep, timeout=50)
    print(response)
    soup = BeautifulSoup(response.content, "html.parser")
    #print(soup)
    ls = []
    lists_single1.append("#")
    for element in soup.findAll("div", {"class":"wordlist-item"}):
        words=[]
        ls.append(element.get_text())
        
        words = ls[-1]
        lists_single1.append(words)
        
            #print(lists[-1])

#print(lists)

https://www.enchantedlearning.com/wordlist/big.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/country.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/compoundwords.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/animal.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/happy.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/adjectives.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/adverbs.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/alliteration.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/amphibian.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/animal.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/opposites.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/aprilfool.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/art.shtml
<Response [200]>
https://www.enchantedlearning.

<Response [200]>
https://www.enchantedlearning.com/wordlist/office.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/opposites.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/duos.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/people.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/adjectivesforpeople.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/pirate.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/plants.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/positivewords.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/postal.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/prepositions.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/presidentsday.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/pronouns.shtml
<Response [200]>
https://www.enchantedlearning.com/wordlist/regularverbs.shtml
<Response

In [17]:
print(lists_single1)

['#', 'adult', 'astronomical', 'boundless', 'brawny', 'broad', 'capacious', 'chunky', 'colossal', 'considerable', 'copious', 'corpulent', 'deep', 'economy-sized', 'elephantine', 'enormous', 'epic', 'expansive', 'extensive', 'extra-large', 'family-sized', 'fat', 'full', 'full-sized', 'gargantuan', 'giant', 'gigantic', 'ginormous', 'goodly', 'grand', 'great', 'heaping', 'heavy', 'hefty', 'Herculean', 'huge', 'hulking', 'humongous', 'husky', 'immense', 'imposing', 'impressive', 'infinite', 'jumbo', 'king-sized', 'large', 'limitless', 'lofty', 'long', 'mammoth', 'massive', 'mega', 'mighty', 'miles-wide', 'monstrous', 'monumental', 'mountainous', 'multifarious', 'multiplicity', 'multitude', 'numerous', 'obese', 'outsized', 'overabundant', 'oversized', 'piled high', 'plentiful', 'plump', 'portly', 'powerful', 'prodigious', 'queen-sized', 'roly-poly', 'sizable', 'spacious', 'stout', 'strapping', 'substantial', 'super', 'super-sized', 'sweeping', 'tall', 'thick', 'thick-set', 'titanic', 'tower

In [18]:
ls_s = lists_single1.copy()

ls_of_ls = [ls_s.split(',') for ls_s in ','.join(ls_s).split('#')]

In [19]:
ls_of_ls

[[''],
 ['',
  'adult',
  'astronomical',
  'boundless',
  'brawny',
  'broad',
  'capacious',
  'chunky',
  'colossal',
  'considerable',
  'copious',
  'corpulent',
  'deep',
  'economy-sized',
  'elephantine',
  'enormous',
  'epic',
  'expansive',
  'extensive',
  'extra-large',
  'family-sized',
  'fat',
  'full',
  'full-sized',
  'gargantuan',
  'giant',
  'gigantic',
  'ginormous',
  'goodly',
  'grand',
  'great',
  'heaping',
  'heavy',
  'hefty',
  'Herculean',
  'huge',
  'hulking',
  'humongous',
  'husky',
  'immense',
  'imposing',
  'impressive',
  'infinite',
  'jumbo',
  'king-sized',
  'large',
  'limitless',
  'lofty',
  'long',
  'mammoth',
  'massive',
  'mega',
  'mighty',
  'miles-wide',
  'monstrous',
  'monumental',
  'mountainous',
  'multifarious',
  'multiplicity',
  'multitude',
  'numerous',
  'obese',
  'outsized',
  'overabundant',
  'oversized',
  'piled high',
  'plentiful',
  'plump',
  'portly',
  'powerful',
  'prodigious',
  'queen-sized',
  'roly

In [20]:
ls_of_ls_df = pd.DataFrame(ls_of_ls)
ls_of_ls_df = ls_of_ls_df.transpose()
ls_of_ls_df = ls_of_ls_df.drop([0])
ls_of_ls_df= ls_of_ls_df.drop([0], axis=1)
ls_of_ls_df.columns =column_names

In [21]:
ls_of_ls_df

,big,country,compoundwords,animal,happy,adjectives,adverbs,alliteration,amphibian,animal,...,water,big,happy,many,said,weapons,weather,measurement,winter,yard
1,adult,Afghanistan,aftereffect,aardvark,amused,abandoned,abnormally,awesome alliterations,amphibian,aardvark,...,aqua,adult,amused,(a) wealth,acknowledged,A-bomb,accumulation,acre,anorak,awning
2,astronomical,Albania,afternoon,abalone,beaming,able,aboard,back-biting,amphiuma,abalone,...,aqueduct,astronomical,beaming,acres,added,ammo,advisory,angstrom,arctic,back porch
3,boundless,Algeria,afterthought,African elephant,blissful,absolute,about,back breaker,army of frogs,African elephant,...,aquifer,boundless,blissful,army,admitted,ammunition,air,area,balaclava,back stoop
4,brawny,Andorra,airbag,African gray parrot,blithe,academic,abroad,put it on the back burner,barking frog,African gray parrot,...,artesian well,brawny,blithe,assortment,advised,armaments,air mass,baker's dozen,below zero,backyard
5,broad,Angola,aircraft,African penguin,buoyant,acceptable,absentmindedly,back to basics,black toad,African penguin,...,basin,broad,buoyant,astronomical amount,affirmed,arms,air pollution,bale,beret,basketball hoop
6,capacious,Antigua and Barbuda,airline,African rock python,carefree,acclaimed,absolutely,bad blood,brook salamander,African rock python,...,black ice,capacious,carefree,batch,agreed,arrow,air pressure,barrel,biting,bench
7,chunky,Argentina,airplane,African wild cat,cheerful,accomplished,abundantly,bare bones,bullfrog,African wild cat,...,blizzard,chunky,cheerful,billions,alleged,assault rifle,almanac,bit,bitter cold,birdbath
8,colossal,Armenia,airport,agouti,cheery,accurate,accidentally,bated breath,caecilian,agouti,...,body of water,colossal,cheery,blizzard,alluded,atom bomb,altocumulus,bolt,black ice,bulbs
9,considerable,Australia,airtight,airedale terrier,chipper,aching,accordingly,beach bum,clawed frog,airedale terrier,...,boil,considerable,chipper,boatload,announced,atomic bomb,altostratus,breadth,blanket,bush
10,copious,Austria,angelfish,Alaskan malamute,chirpy,acidic,actively,bear the brunt of,climbing salamander,Alaskan malamute,...,brackish water,copious,chirpy,boundless amount,answered,autocannon,anemometer,bulk,bleak,carport


In [22]:
ls_of_ls_df_clean = ls_of_ls_df.loc[:,~ls_of_ls_df.columns.duplicated()]

In [23]:
ls_of_ls_df_clean = ls_of_ls_df_clean.reindex(sorted(ls_of_ls_df_clean.columns), axis=1)

ls_of_ls_df_clean

,adjectives,adjectivesforpeople,adverbs,alliteration,amphibian,animal,aprilfool,art,astronomy,baseball,...,vacation,valentine,vegetables,verbs,virtues,water,weapons,weather,winter,yard
1,abandoned,able,abnormally,awesome alliterations,amphibian,aardvark,absurd,abstract,albedo,American League,...,airfare,admirer,acorn squash,accept,accountability,aqua,A-bomb,accumulation,anorak,awning
2,able,abnormal,aboard,back-biting,amphiuma,abalone,absurdity,acrylic paint,Alpha Centauri,assist,...,airplane,adoration,alfalfa,ache,affection,aqueduct,ammo,advisory,arctic,back porch
3,absolute,above average,about,back breaker,army of frogs,African elephant,All Fools' Day,airbrush,apastron,at bat,...,airport,adore,artichoke,acknowledge,agreeableness,aquifer,ammunition,air,balaclava,back stoop
4,academic,absent-minded,abroad,put it on the back burner,barking frog,African gray parrot,amuse,animation,aperture,athlete,...,amusement park,affection,arugula,act,amiability,artesian well,armaments,air mass,below zero,backyard
5,acceptable,adventurous,absentmindedly,back to basics,black toad,African penguin,amusing,architecture,aphelion,ball,...,automobile,affectionate,asparagus,add,benevolence,basin,arms,air pollution,beret,basketball hoop
6,acclaimed,affectionate,absolutely,bad blood,brook salamander,African rock python,April,art,apogee,base,...,backpack,amorous,avocado,admire,bravery,black ice,arrow,air pressure,biting,bench
7,accomplished,agile,abundantly,bare bones,bullfrog,African wild cat,April Fool's Day,art gallery,asterism,baseball,...,baggage,angel,bamboo shoots,admit,care,blizzard,assault rifle,almanac,bitter cold,birdbath
8,accurate,agreeable,accidentally,bated breath,caecilian,agouti,celebrate,artist,asteroid,baserunning,...,bags,balloons,basil,admonish,caring,body of water,atom bomb,altocumulus,black ice,bulbs
9,aching,alert,accordingly,beach bum,clawed frog,airedale terrier,chicanery,artist's bridge,astronaut,bases loaded,...,bathing suit,be mine,beans,adopt,charity,boil,atomic bomb,altostratus,blanket,bush
10,acidic,amazing,actively,bear the brunt of,climbing salamander,Alaskan malamute,childish,artistic,astronomer,bat,...,beach,beau,beets,advise,cleanliness,brackish water,autocannon,anemometer,bleak,carport


### Save the word list for later use to skip the scraping next time

In [24]:
ls_of_ls_df_clean.to_csv(r'wordlist.csv', sep=';', header=True, index=False)

# Generator app
This part can be executed after the word list is generated. It does not rely on the previous code being run again.

It only needs the word list created by the webscraper.

In [25]:
import pandas as pd
import numpy as np
from os import path
import csv
import random
from random import choice

from IPython.display import clear_output # clear user input, refresh image

### Load word list 

In [26]:
# load most recent table
words_df = pd.read_csv("wordlist.csv", sep=";", index_col=0)
words_df.head()

,adjectivesforpeople,adverbs,alliteration,amphibian,animal,aprilfool,art,astronomy,baseball,bathroom,...,vacation,valentine,vegetables,verbs,virtues,water,weapons,weather,winter,yard
adjectives,,,,,,,,,,,,,,,,,,,,,
abandoned,able,abnormally,awesome alliterations,amphibian,aardvark,absurd,abstract,albedo,American League,antiseptic,...,airfare,admirer,acorn squash,accept,accountability,aqua,A-bomb,accumulation,anorak,awning
able,abnormal,aboard,back-biting,amphiuma,abalone,absurdity,acrylic paint,Alpha Centauri,assist,aspirin,...,airplane,adoration,alfalfa,ache,affection,aqueduct,ammo,advisory,arctic,back porch
absolute,above average,about,back breaker,army of frogs,African elephant,All Fools' Day,airbrush,apastron,at bat,atomizer,...,airport,adore,artichoke,acknowledge,agreeableness,aquifer,ammunition,air,balaclava,back stoop
academic,absent-minded,abroad,put it on the back burner,barking frog,African gray parrot,amuse,animation,aperture,athlete,bandages,...,amusement park,affection,arugula,act,amiability,artesian well,armaments,air mass,below zero,backyard
acceptable,adventurous,absentmindedly,back to basics,black toad,African penguin,amusing,architecture,aphelion,ball,basin,...,automobile,affectionate,asparagus,add,benevolence,basin,arms,air pollution,beret,basketball hoop


In [27]:
# show what categories are available
list(words_df.columns) 

['adjectivesforpeople',
 'adverbs',
 'alliteration',
 'amphibian',
 'animal',
 'aprilfool',
 'art',
 'astronomy',
 'baseball',
 'bathroom',
 'beach',
 'big',
 'biomes',
 'birds',
 'birthday',
 'boat',
 'bodiesofwater',
 'body',
 'buildings',
 'camping',
 'car',
 'carnival',
 'carparts',
 'castle',
 'cats',
 'chinesenewyear',
 'christmas',
 'circus',
 'clothes',
 'colors',
 'columbus',
 'compoundwords',
 'computer',
 'conjunctions',
 'constitution',
 'container',
 'cooking',
 'cookingtools',
 'country',
 'dance',
 'dentist',
 'desserts',
 'doctor',
 'dogs',
 'dolch',
 'driving',
 'duos',
 'election',
 'elements',
 'emotions',
 'energy',
 'fall',
 'family',
 'farm',
 'fire',
 'fish',
 'flagday',
 'flowers',
 'food',
 'foodweb',
 'fruit',
 'furniture',
 'geography',
 'goodluck',
 'grammar',
 'groundhogday',
 'halloween',
 'happiness',
 'happy',
 'hats',
 'herbs',
 'holidays',
 'house',
 'householddevices',
 'housing',
 'insect',
 'interjections',
 'irregularverbs',
 'jobs',
 'kitchen',
 '

#### Remove less useful categories 

In [28]:
drop_columns = ['adverbs', 'aprilfool',
                 'conjunctions', 
                'grammar','interjections',
                'irregularverbs', 'longe',
                'longu','nounandverb', 'prepositions', 'pronouns',
                'regularverbs', 'said']

#### see see the data

In [29]:
words_df_clean=words_df.copy()
words_df_clean= words_df_clean.drop(drop_columns, axis=1)
words_df_clean

,adjectivesforpeople,alliteration,amphibian,animal,art,astronomy,baseball,bathroom,beach,big,...,vacation,valentine,vegetables,verbs,virtues,water,weapons,weather,winter,yard
adjectives,,,,,,,,,,,,,,,,,,,,,
abandoned,able,awesome alliterations,amphibian,aardvark,abstract,albedo,American League,antiseptic,barnacle,adult,...,airfare,admirer,acorn squash,accept,accountability,aqua,A-bomb,accumulation,anorak,awning
able,abnormal,back-biting,amphiuma,abalone,acrylic paint,Alpha Centauri,assist,aspirin,bathing suit,astronomical,...,airplane,adoration,alfalfa,ache,affection,aqueduct,ammo,advisory,arctic,back porch
absolute,above average,back breaker,army of frogs,African elephant,airbrush,apastron,at bat,atomizer,bay,boundless,...,airport,adore,artichoke,acknowledge,agreeableness,aquifer,ammunition,air,balaclava,back stoop
academic,absent-minded,put it on the back burner,barking frog,African gray parrot,animation,aperture,athlete,bandages,beach,brawny,...,amusement park,affection,arugula,act,amiability,artesian well,armaments,air mass,below zero,backyard
acceptable,adventurous,back to basics,black toad,African penguin,architecture,aphelion,ball,basin,beachball,broad,...,automobile,affectionate,asparagus,add,benevolence,basin,arms,air pollution,beret,basketball hoop
acclaimed,affectionate,bad blood,brook salamander,African rock python,art,apogee,base,bath,bikini,capacious,...,backpack,amorous,avocado,admire,bravery,black ice,arrow,air pressure,biting,bench
accomplished,agile,bare bones,bullfrog,African wild cat,art gallery,asterism,baseball,bath mat,boardwalk,chunky,...,baggage,angel,bamboo shoots,admit,care,blizzard,assault rifle,almanac,bitter cold,birdbath
accurate,agreeable,bated breath,caecilian,agouti,artist,asteroid,baserunning,bath towel,boat,colossal,...,bags,balloons,basil,admonish,caring,body of water,atom bomb,altocumulus,black ice,bulbs
aching,alert,beach bum,clawed frog,airedale terrier,artist's bridge,astronaut,bases loaded,bathrobe,boogie board,considerable,...,bathing suit,be mine,beans,adopt,charity,boil,atomic bomb,altostratus,blanket,bush


In [30]:
#show how many words per category are available
words_df_clean.isna().sum()

adjectivesforpeople    1139
alliteration           1269
amphibian              1448
animal                  716
art                    1346
astronomy              1312
baseball               1426
bathroom               1432
beach                  1410
big                    1436
biomes                 1430
birds                  1283
birthday               1453
boat                   1315
bodiesofwater          1430
body                   1369
buildings              1334
camping                1462
car                    1483
carnival               1432
carparts               1357
castle                 1439
cats                   1470
chinesenewyear         1433
christmas              1336
circus                 1420
clothes                1257
colors                 1414
columbus               1431
compoundwords             1
                       ... 
roadways               1470
rocks                  1412
rooms                  1409
school                 1416
science             

In [31]:
words_df_clean=words_df_clean.fillna("no_word")

## Random name generator

In [32]:
name_suggestions = []

In [33]:
import random

print("The following categories are available:")
print()
print(list(words_df_clean.columns))
print()

for i in range(5): #generate 5 examples
    print("Choose your first category.")
    first_col = input()
    first_list = words_df_clean[first_col].tolist()
    first_list_new = [c for c in first_list if c != 'no_word']
    first_word = random.choice(first_list_new)
    
   # print(first_word)   
    #print()
       
    print("Choose your second category.")
    second_col = input()    
    second_list = words_df_clean[second_col].tolist()
    second_list_new = [c for c in second_list if c != 'no_word']
    second_word = random.choice(second_list_new)        

    #print(second_word)
    #print()
    print()
    print("~~~~~")
    print()
    print("The new name is:")          
    print(first_word + " " + second_word)
    
    name_suggestions.append(first_word + " " + second_word)
    
    print()
    print("*****")
    print()

The following categories are available:

['adjectivesforpeople', 'alliteration', 'amphibian', 'animal', 'art', 'astronomy', 'baseball', 'bathroom', 'beach', 'big', 'biomes', 'birds', 'birthday', 'boat', 'bodiesofwater', 'body', 'buildings', 'camping', 'car', 'carnival', 'carparts', 'castle', 'cats', 'chinesenewyear', 'christmas', 'circus', 'clothes', 'colors', 'columbus', 'compoundwords', 'computer', 'constitution', 'container', 'cooking', 'cookingtools', 'country', 'dance', 'dentist', 'desserts', 'doctor', 'dogs', 'dolch', 'driving', 'duos', 'election', 'elements', 'emotions', 'energy', 'fall', 'family', 'farm', 'fire', 'fish', 'flagday', 'flowers', 'food', 'foodweb', 'fruit', 'furniture', 'geography', 'goodluck', 'groundhogday', 'halloween', 'happiness', 'happy', 'hats', 'herbs', 'holidays', 'house', 'householddevices', 'housing', 'insect', 'jobs', 'kitchen', 'landforms', 'languages', 'leaders', 'legal', 'mammal', 'many', 'math', 'measurement', 'metals', 'military', 'mlk', 'money', '

In [34]:
#the name suggestions will be saved over mutiple rounds
name_suggestions

['parsley axis',
 'hot chocolate righteousness',
 'workstation umbra',
 'wind vane communication',
 'palace frame']

In [35]:
with open("name_suggestions.csv", "a") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in name_suggestions:
        writer.writerow([val])